## DSC 540 - Week 12 - 12.3 Final Project
### David Kinney

The following is due submitted to the assignment link or submit a link to your GitHub repository to the assignment link:  

* Your formatted dataset with at least 15-20 variables (if the API or Webpage you selected doesn’t have that many fields available on it, you will want to search again, or do multiple!)
* Your code or screenshots of your code outlining the steps and process you had to take to pull data from the API or web page and the steps you took to format the data.
* 2 Data Transformation/Clean-up Steps (can be any that we learned in class)
* A 250-word paper summarizing your steps and any challenges you ran into during the project. Discuss the importance and relevance of this type of process if you were a data scientist. How often do you think you would have to do this to get the data you need?
*****************************************

In [1]:
import json
import pandas as pd
import requests

from collections import ChainMap

Using **Open Weather Map**, retrieve most recent 5 day/3 hour weather history for the city of Atlanta.

In [2]:
# Locate Atlanta's ID in city.list.json
with open('city.list.json', encoding='utf8') as json_file:
    cities = json.load(json_file)
    
next(item for item in cities if item["name"] == "Atlanta")

{'id': 4180439,
 'name': 'Atlanta',
 'country': 'US',
 'coord': {'lon': -84.387978, 'lat': 33.749001}}

In [3]:
# Retrieve 5-day, 3-hour weather history from openweathermap.org
mykey = '0078c09e9d78e416c1495e4160e0cd00'
atlid = '4180439'
api_call = 'http://api.openweathermap.org/data/2.5/forecast?id={}&APPID={}'.format(atlid, mykey)

response = requests.get(api_call)
weather_json = json.loads(response.text)
atlanta_weather = weather_json['list']
df_weather = pd.DataFrame(atlanta_weather)

In [4]:
df_weather.head()

,dt,main,weather,clouds,wind,sys,dt_txt,rain
0,1573506000,"{'temp': 292.08, 'temp_min': 290.86, 'temp_max...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 78},"{'speed': 2.55, 'deg': 216}",{'pod': 'd'},2019-11-11 21:00:00,NaN
1,1573516800,"{'temp': 287, 'temp_min': 286.08, 'temp_max': ...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 88},"{'speed': 2.24, 'deg': 207}",{'pod': 'n'},2019-11-12 00:00:00,NaN
2,1573527600,"{'temp': 287.1, 'temp_min': 286.49, 'temp_max'...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 100},"{'speed': 1.99, 'deg': 207}",{'pod': 'n'},2019-11-12 03:00:00,NaN
3,1573538400,"{'temp': 286.29, 'temp_min': 285.98, 'temp_max...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 100},"{'speed': 1.88, 'deg': 189}",{'pod': 'n'},2019-11-12 06:00:00,{'3h': 1.44}
4,1573549200,"{'temp': 286.61, 'temp_min': 286.61, 'temp_max...","[{'id': 501, 'main': 'Rain', 'description': 'm...",{'all': 100},"{'speed': 3.09, 'deg': 282}",{'pod': 'n'},2019-11-12 09:00:00,{'3h': 9.69}


### Transformation and Cleaning

* Drop **dt** and **sys** columns.
* Extract **main**, **weather**, **clouds**, **wind** and **rain** from dictionaries into separate columns.
* Convert temperatures from Kelvin to Fahrenheit.
* Rename columns to more user friendly names.
* Evaluate missing data and clean accordingly.
***********************************

In [5]:
df_weather_clean = df_weather.drop(['dt', 'sys'], axis=1)

In [6]:
df_weather_clean = pd.concat([df_weather_clean.drop(['main'], axis=1), 
                              df_weather_clean['main'].apply(pd.Series)], axis=1)
df_weather_clean = pd.concat([df_weather_clean.drop(['clouds'], axis=1), 
                              df_weather_clean['clouds'].apply(pd.Series)], axis=1)
df_weather_clean = pd.concat([df_weather_clean.drop(['wind'], axis=1), 
                              df_weather_clean['wind'].apply(pd.Series)], axis=1)
df_weather_clean = pd.concat([df_weather_clean.drop(['rain'], axis=1), 
                              df_weather_clean['rain'].apply(pd.Series)], axis=1)

In [7]:
df_weather_clean.head()

,weather,dt_txt,temp,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,all,speed,deg,0,3h
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",2019-11-11 21:00:00,292.08,290.86,292.08,1017.0,1017.0,984.0,53.0,1.22,78,2.55,216.0,NaN,NaN
1,"[{'id': 804, 'main': 'Clouds', 'description': ...",2019-11-12 00:00:00,287.00,286.08,287.00,1018.0,1018.0,985.0,72.0,0.92,88,2.24,207.0,NaN,NaN
2,"[{'id': 804, 'main': 'Clouds', 'description': ...",2019-11-12 03:00:00,287.10,286.49,287.10,1017.0,1017.0,984.0,77.0,0.61,100,1.99,207.0,NaN,NaN
3,"[{'id': 500, 'main': 'Rain', 'description': 'l...",2019-11-12 06:00:00,286.29,285.98,286.29,1015.0,1015.0,982.0,92.0,0.31,100,1.88,189.0,NaN,1.44
4,"[{'id': 501, 'main': 'Rain', 'description': 'm...",2019-11-12 09:00:00,286.61,286.61,286.61,1014.0,1014.0,981.0,97.0,0.00,100,3.09,282.0,NaN,9.69


In [8]:
# temp and temp_min appear to be identical, so dropping temp
df_weather_clean.drop(['temp'], axis=1, inplace=True)

In [9]:
# Not sure why they broke from consistency, but "weather" is a list containing
# one dictionary item. This is probably not the most efficient approach, but in
# order to do away with the list type, I create one big list of all the dictionary
# items, convert that to a pandas dataframe, and then contcatenate that, column-wise,
# to the clean dataset.
w = []
for L in df_weather_clean['weather']:
    data = dict(ChainMap(*L))
    w.append(data)
    
df_w = pd.DataFrame(w)
df_weather_clean = pd.concat([df_weather_clean, df_w], axis=1)
df_weather_clean.drop(['weather', 'icon'], axis=1, inplace=True)

In [10]:
df_weather_clean.head()

,dt_txt,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,all,speed,deg,0,3h,id,main,description
0,2019-11-11 21:00:00,290.86,292.08,1017.0,1017.0,984.0,53.0,1.22,78,2.55,216.0,NaN,NaN,803,Clouds,broken clouds
1,2019-11-12 00:00:00,286.08,287.00,1018.0,1018.0,985.0,72.0,0.92,88,2.24,207.0,NaN,NaN,804,Clouds,overcast clouds
2,2019-11-12 03:00:00,286.49,287.10,1017.0,1017.0,984.0,77.0,0.61,100,1.99,207.0,NaN,NaN,804,Clouds,overcast clouds
3,2019-11-12 06:00:00,285.98,286.29,1015.0,1015.0,982.0,92.0,0.31,100,1.88,189.0,NaN,1.44,500,Rain,light rain
4,2019-11-12 09:00:00,286.61,286.61,1014.0,1014.0,981.0,97.0,0.00,100,3.09,282.0,NaN,9.69,501,Rain,moderate rain


In [11]:
# Convert the temperatures from Kelvin to Fahrenheit
df_weather_clean['temp_min'] = df_weather_clean['temp_min'].apply(lambda x: (x - 273.15) * 9 / 5 + 32)
df_weather_clean['temp_max'] = df_weather_clean['temp_max'].apply(lambda x: (x - 273.15) * 9 / 5 + 32)

In [12]:
# Rename a few of the columns to be more user friendly.
df_weather_clean.rename(columns={"dt_txt": "datetime", "all": "clouds", "speed": "windspeed", "deg": "heading", "3h": "rain"}, inplace=True)

In [13]:
# I suspect the column with a 0 column name is bogus. Check to see if anything is in it. 
# If not, drop it...
df_weather_clean.iloc[:, [11]].isna().all()

0    True
dtype: bool

In [14]:
df_weather_clean.drop(df_weather_clean.iloc[:, [11]], axis=1, inplace=True)

In [15]:
df_weather_clean.head()

,datetime,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,clouds,windspeed,heading,rain,id,main,description
0,2019-11-11 21:00:00,63.878,66.074,1017.0,1017.0,984.0,53.0,1.22,78,2.55,216.0,NaN,803,Clouds,broken clouds
1,2019-11-12 00:00:00,55.274,56.930,1018.0,1018.0,985.0,72.0,0.92,88,2.24,207.0,NaN,804,Clouds,overcast clouds
2,2019-11-12 03:00:00,56.012,57.110,1017.0,1017.0,984.0,77.0,0.61,100,1.99,207.0,NaN,804,Clouds,overcast clouds
3,2019-11-12 06:00:00,55.094,55.652,1015.0,1015.0,982.0,92.0,0.31,100,1.88,189.0,1.44,500,Rain,light rain
4,2019-11-12 09:00:00,56.228,56.228,1014.0,1014.0,981.0,97.0,0.00,100,3.09,282.0,9.69,501,Rain,moderate rain


In [16]:
df_weather_clean.tail()

,datetime,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,clouds,windspeed,heading,rain,id,main,description
35,2019-11-16 06:00:00,41.900,41.900,1021.0,1021.0,987.0,67.0,0.0,0,2.04,355.0,NaN,800,Clear,clear sky
36,2019-11-16 09:00:00,41.666,41.666,1021.0,1021.0,987.0,75.0,0.0,0,2.01,350.0,NaN,800,Clear,clear sky
37,2019-11-16 12:00:00,40.568,40.568,1022.0,1022.0,988.0,78.0,0.0,0,1.82,359.0,NaN,800,Clear,clear sky
38,2019-11-16 15:00:00,50.720,50.720,1023.0,1023.0,989.0,53.0,0.0,0,2.86,47.0,NaN,800,Clear,clear sky
39,2019-11-16 18:00:00,56.912,56.912,1021.0,1021.0,988.0,40.0,0.0,0,2.95,64.0,NaN,800,Clear,clear sky


<center><h2>Summary Paper</h2></center>

***************************************************

I'm drawn to investigating climate change, so when I saw that one of the suggested APIs was the **Open Weather Map** site I zeroed in. Unfortunately, long-term historical data requires a paid membership, but for this exercise I felt that using the free subscription and utilizing a 5-day, 3-hour interval dataset of my hometown of Atlanta would be applicable for the exercise.

The first step was finding the **city ID** for Atlanta. Open Weather Map has a downloadable JSON file with all the cities and ID's. I downloaded that and searched for `name='Atlanta'` to retrieve the ID. After registering for a free API key, I had everything I needed to send a request to the API to get the weather data.

**Transformation and Cleaning**

After retrieving the dataset I displayed the first few rows. Initial exploration revealed a few items that could be immediately addressed:

* There were some columns that were not relevant to analysis; I dropped them.
* A number of columns were themselves JSON objects (in one case, a list of a dictionary item). I expanded those out into individual columns.
* Temperatures were in Kelvin; I converted them to Fahrenheit.
* Replace some column names with more user-friendly titles.
* Some columns had NaN values, which might bear further investigation.

This Jupyter notebook shows the sequence of steps I took to perform this process, as well as others that cropped up along the way.

**Importance and Relevance**

In the few years I've been doing data analysis and science, I have yet to come across a dataset that is clean and ready for use "out of the box". In this particular use case, being able to use those columns that were JSON objects would be virtually impossible. Converting each key to its own column made the data much more understandable. I don't feel that it's an exaggeration to say that data preparation is required in virtually every Data Science project, whether on a large scale or small.